In [2]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here’s a helpful package to load
library(tidyverse) # metapackage of all tidyverse packages
# Input data files are available in the read-only “../input/” directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
list.files(path = '../input')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using “Save & Run All”
# You can also write temporary files to /kaggle/temp/, but they won’t be saved outside of the current session

In [3]:
install.packages('rpart')
install.packages('rpart.plot')
install.packages('gbm')
install.packages('randomForest')
install.packages('nnet')

In [5]:
library('rpart')
library('rpart.plot')
library('randomForest')
library('gbm')
library('nnet')

In [6]:

dfTrain <- read.csv('/kaggle/input/examen-ml2/train.csv', sep = ';')
dfTest <- read.csv('/kaggle/input/examen-ml2/test.csv', sep = ';')

In [ ]:
summary(dfTrain)

In [8]:

#dfTest[36] <- lapply(dfTest[36], as.factor)
dfTrain[36] <- lapply(dfTrain[36], as.factor)

In [9]:
sum(is.na(dfTrain))
sum(is.na(dfTest))

In [78]:
# glm(y~x) 
#X_train=dfTrain[,-36]
X_train= select(dfTrain,-c(1,36))
y_train=dfTrain[,36]

fit <- rpart(y_train~.,data=X_train)
rpart.plot(fit,extra=1)

In [79]:
confMat<-table(predict(fit, X_train, type = "class"), y_train)
precision<-sum(diag(confMat))/sum(confMat)
# diag (succé, les reussites)
# doit être le plus grand possible
print(confMat)
print(1-precision)

In [80]:
fit.max <- rpart(y_train~.,data=X_train ,control = rpart.control(cp=0, minsplit = 2, minbucket = 1))
plotcp(fit.max)

In [81]:
printcp(fit.max)

best : 12 0.00290698     28  0.393411 0.50097 0.019285

In [84]:
# on prend le meilleur cp précédent
tree<-prune(fit.max,cp=0.00290698)
#insert code
#X_test=dfTest[,-36]
X_test= select(dfTest,-c(1))

rpart.plot(tree,extra=1)
# récupération du prédict
y_test <- predict(tree, X_test, type = "class")

#print(y_test)

In [ ]:
class(y_test)

In [ ]:
library(dplyr)
dfTest$Target <- y_test
output1 <- dfTest %>%
    select(Student.ID, Target) 

colnames(output1) <- c("StudentId", "Target")
output1$Target <- as.character(output1$Target)

output1 %>%
    write.csv("output23.csv", row.names=FALSE, quote = FALSE)



In [ ]:
#y_test <- as.data.frame(y_test)
#y_test <- cbind(StudentId = rownames(y_test), y_test)
#rownames(y_test) <- NULL
#colnames(y_test) <- c("StudentId","Target")
#y_test$StudentId <- as.numeric(y_test$StudentId)
#y_test"""


In [ ]:
#write.csv(y_test,'sortie42.csv', row.names = F)

## Forêt Aléatoire

In [94]:
error_rate<-c()
error_rate_oo<-c()
# pour 21 facteurs
p=34
Ntrees=c(1,30,50,100,150,200,300,500,1000,5000)
for( m in c(sqrt(p),p/2)){
    for(i in Ntrees){
        fit.rf <- randomForest(y_train~., data=X_train,xtest=X_test,ytest=y_test,ntree=i ,mtry=m,maxnodes=30, importance=TRUE)
        # calcul confiance Matrice
        cm<-fit.rf$confusion
        # 1-sum(diag(cm))/sum(cm[,-3]) -> bloc error rate (ici on elever la colone 4 qui est l'erreur)
        error_rate_oo<-c(error_rate_oo,1-sum(diag(cm))/sum(cm[,-4]))
        confMat<-fit.rf$test$confusion
        #1-sum(diag(confMat))/sum(confMat[,-3])
        error_rate<-c(error_rate,1-sum(diag(confMat))/sum(confMat[,-4]))
    }
}
l=length(Ntrees)
plot(Ntrees,error_rate[(1:l)],type='l',col='blue',lwd=2,ylim=c(min(error_rate)-0.1,max(error_rate)+0.1),ylab="Error rate",xlab="Number of Trees")
points(Ntrees,error_rate_oo[(1:l)],type='l',col='blue',lty=2,lwd=2)
points(Ntrees,error_rate[((1+l):(2*l))],type='l',col='red',lwd=2,ylab="Error rate",xlab="Number of Trees")
points(Ntrees,error_rate_oo[((1+l):(2*l))],type='l',col='red',lty=2,lwd=2)
legend("topright", legend=c("m=sqrt(p)","m=sqrt(p),OOB","m=p/2","m=p/2,OOB"),lty = c(1,2,1,2),col=c("blue","blue","red","red"))

In [86]:
fit.rf <- randomForest(y_train ~ ., data = X_train, xtest = X_test, ytest = y_test, ntree = 1000, mtry = p/2, maxnodes = 30, importance = TRUE)
fit.rf $importance
varImpPlot(fit.rf)

Ok maintenent on refait un modèles avec les variables qui arrivent en tête

In [88]:
dfTrain2 <- select(dfTrain,'Student.ID','Curricular.units.2nd.sem..approved.','Tuition.fees.up.to.date','Curricular.units.2nd.sem..grade.','Curricular.units.1st.sem..approved.','Curricular.units.2nd.sem..enrolled.','Curricular.units.1st.sem..enrolled.','Age.at.enrollment','Course','Target')
X_train2= select(dfTrain2,-c(1,10))
y_train2=dfTrain2[,10]

fit2 <- rpart(y_train2~.,data=X_train2)
rpart.plot(fit2,extra=1)

In [89]:
confMat2<-table(predict(fit2, X_train2, type = "class"), y_train2)
precision2<-sum(diag(confMat2))/sum(confMat2)
# diag (succé, les reussites)
# doit être le plus grand possible
print(confMat2)
print(1-precision2)

In [90]:
fit.max2 <- rpart(y_train2~.,data=X_train2 ,control = rpart.control(cp=0, minsplit = 2, minbucket = 1))
plotcp(fit.max2)

In [91]:
printcp(fit.max2)

Best : 10 0.00258398     23  0.430233 0.48450 0.019061

In [92]:
#dfTest2 = dfTest.columns.drop(['Educational.special.needs','Daytime.evening.attendance',])
dfTest2 <- select(dfTest,'Student.ID','Curricular.units.2nd.sem..approved.','Tuition.fees.up.to.date','Curricular.units.2nd.sem..grade.','Curricular.units.1st.sem..approved.','Curricular.units.2nd.sem..enrolled.','Curricular.units.1st.sem..enrolled.','Age.at.enrollment','Course')
X_test2= select(dfTest2,-c(1))

# on met cp
tree2<-prune(fit.max2,cp=0.00258398)

rpart.plot(tree2,extra=1)
# récupération du prédict
y_test2 <- predict(tree2, X_test2, type = "class")

In [93]:
dfTest2$Target <- y_test2
output2 <- select(dfTest2,'Student.ID', 'Target') 

colnames(output2) <- c("StudentId", "Target")

output2 %>%
    write.csv("Model3.csv", row.names=FALSE)
